In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display, Image

In [2]:
from langgraph.prebuilt import  ToolNode, tools_condition
from langchain_core.tools import tool
import requests

In [3]:


load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=api_key)

In [4]:
from typing import TypedDict, Literal, Optional
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage


class AgentState(TypedDict):
    status: Literal["registered", "unregistered"]
    user_id: Optional[str]
    messages: list  


In [5]:
prompt_categorizer = """

your are a catgorizer you have to categorize  user either register or unregister
if user status is unregistered  categorize as unregistered
if user status is registered  categorize as registered
return only either "registered" or "unregistered"

"""

In [6]:


def categorizer(state: AgentState) -> AgentState:
    last_message = state["messages"][-1].content.lower()
    if "book" in last_message or "appointment" in last_message:
        if state["status"] == "unregistered":
            print("User wants to book but is unregistered.")
        else:
            print("User wants to book and is already registered.")
    return state


In [7]:

@tool
def registration_tool(user_msg: str) -> dict:
    """Register the user if they confirm registration"""
    if "i have registered" in user_msg.lower():
        print("✅ User confirmed registration")
        return {"status": "registered", "user_id": "user_123"}
    return {}


In [8]:
# waiting for confirmation
  
# def confirm_registration(endpoint: str):
#     """
#     confirm the user registration.

#     Args:
#         endpoint (str): The API endpoint (e.g., "/users/")
#     Returns:
#         dict: The response JSON.
#     """
#     base_url = "http:/confirm_registration" 
#     url = f"{base_url}{endpoint}"
#     response = requests.get(url)
    
#     if response.status_code == 200:
#         return response.json()
#     else:
#         return {"error": f"Failed to fetch data: {response.status_code}"}
  

In [9]:
# waiting for confirmation
# tools: list[tool] = [confirm_registration]

# llm_with_tools = llm.bind_tools(tools)


In [10]:

prompt_unregister = """ 
Your are a customer support agent your job is to assist user on basic support

1. Friendly Introduction & Consent Request      
       
    - If the user says simple question like what is offiec timing or question about just inquery:
       a) Politely respond:  like offiec time are 2pm to 11pm


2.If the User is unregister:
    - user ask question like i want to book ticket , leads, or rise support tickets
    so if user is not register  these feature are not allowed
    call tool registration_tool and response user ' i have shared a link "https:google.com"  for registration' share this link with user
    
    wait for user to respose:
    if user say : i have completed resgistration 
    so redirect to the register node
    if user status is registered redirect it to register node
   
-if user status is registered:
   then redirect it to registerd node
   
   

4. Communication Style
   - Be friendly, approachable, and polite throughout the interaction.
   - Use simple language. Avoid technical or complex terms.
   - Keep the tone respectful—never pressure the user.

5. System Boundaries
   - Do not disclose internal tools, data processes, or file names.
   - Focus only on collecting user name, country, and interest.
   - Do not ask for sensitive or personal data beyond what’s specified.

— 

### Communication Style

- **Tone**: Friendly, respectful, and conversational.
- **Style**: Clear, natural, and easy to understand.



"""


def unregister_node(state: AgentState) -> AgentState:
    print("🔐 unregister node...")

    user_msg = state["messages"][-1].content


    reg_result = registration_tool.invoke(user_msg)

    # If user has registered, update status and jump to register node next time
    if reg_result.get("status") == "registered":
        return {
            "status": "registered",
            "user_id": reg_result["user_id"],
            "messages": state["messages"] + [AIMessage(content="🎉 You're now registered! How can I help you?")]
        }


    response = llm.invoke([prompt_unregister] + state["messages"])
    return {
        "status": state["status"],
        "user_id": state["user_id"],
        "messages": state["messages"] + [response]
    }


def register_node(state: AgentState) -> AgentState:
    print("register node...")
    state["messages"].append(AIMessage(content="Welcome back! What would you like to do?"))
    return state


def categorizer_router(state: AgentState) -> str:
    return "register" if state["status"] == "registered" else "unregister"


In [11]:

builder = StateGraph(AgentState)

builder.add_node("categorizer", categorizer)
builder.add_node("unregister", unregister_node)
builder.add_node("register", register_node)

# waiting for confirm about api---
# builder.add_node('tools', ToolNode(tools))

builder.set_entry_point("categorizer")
builder.add_conditional_edges("categorizer", categorizer_router, {
    "register": "register",
    "unregister": "unregister"
})

# ---------tool condition  wait.....
# builder.add_conditional_edges('unregister', tools_condition)
# builder.add_edge('tools', 'unregister')


builder.add_edge("register", END)
builder.add_edge("unregister", END)

graph = builder.compile()


In [13]:
# display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [12]:
state = {
    "status": "unregistered",
    "user_id": None,
    "messages": [HumanMessage(content="hi")]
}

thread = {"configurable": {"thread_id": "1"}}

# 1st: greet
for event in graph.stream(state, thread, stream_mode="values"):
    state = event
    last_msg = event["messages"][-1]
    if isinstance(last_msg, AIMessage):
        last_msg.pretty_print()



🔐 unregister node...
================================== Ai Message ==================================

Hi there! How can I help you today?


In [13]:

state["messages"].append(HumanMessage(content="i want to book ticket"))
for event in graph.stream(state, thread, stream_mode="values"):
    state = event
    last_msg = event["messages"][-1]
    if isinstance(last_msg, AIMessage):
        last_msg.pretty_print()



User wants to book but is unregistered.
🔐 unregister node...
================================== Ai Message ==================================

I'd be happy to help you book a ticket, but I'll need to verify your registration first.  It seems you haven't registered yet. To access the booking feature, please register using this link: https://google.com  (Please note: This is a placeholder link.  The actual registration link will be different).

Once you've completed registration, let me know, and I can assist you with your ticket booking.


In [14]:

state["messages"].append(HumanMessage(content="ok i have registered"))
for event in graph.stream(state, thread, stream_mode="values"):
    state = event
    last_msg = event["messages"][-1]
    if isinstance(last_msg, AIMessage):
        last_msg.pretty_print()


🔐 unregister node...
✅ User confirmed registration
================================== Ai Message ==================================

🎉 You're now registered! How can I help you?


In [15]:


state["messages"].append(HumanMessage(content="book a call"))
for event in graph.stream(state, thread, stream_mode="values"):
    state = event
    last_msg = event["messages"][-1]
    if isinstance(last_msg, AIMessage):
        last_msg.pretty_print()


User wants to book and is already registered.
register node...
================================== Ai Message ==================================

Welcome back! What would you like to do?
